In [2]:
import os, sys
import json
import nltk 
import csv
import random
import nltk  # $ pip install nltk
from nltk.stem import PorterStemmer
from nltk.corpus import cmudict  # >>> nltk.download('cmudict')
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker

from beliefbank_data.utils import generate_assertion, generate_question

In [3]:
%ls beliefbank_data

calibration_facts.json     convert_to_sentences.py  silver_facts.json
calibration_questions.csv  dev_entities.txt         silver_questions.csv
calibration_sentences.csv  eval_entities.txt        silver_sentences.csv
constraints_v2.json        __pycache__/             utils.py


In [4]:
constraints_path = "beliefbank_data/constraints_v2.json"
facts_path = "beliefbank_data/silver_facts.json"

In [5]:
constraints = json.load(open(constraints_path))
facts = json.load(open(facts_path))

In [5]:
facts['american bison']['IsA,mammal']

'yes'

In [6]:
constraints['nodes'][:2]

[{'id': 'IsA,tree'}, {'id': 'IsA,living thing'}]

In [7]:
constraints['links'][:2]

[{'weight': 'yes_yes',
  'direction': 'forward',
  'score': 10,
  'source': 'IsA,tree',
  'target': 'IsA,living thing'},
 {'weight': 'yes_yes',
  'direction': 'forward',
  'score': 10,
  'source': 'IsA,tree',
  'target': 'CapableOf,grow'}]

In [40]:
facts['american bison']

{'IsA,mammal': 'yes',
 'IsA,american bison': 'yes',
 'IsA,animal': 'yes',
 'IsA,vertebrate': 'yes',
 'IsA,warm blooded animal': 'yes',
 'CapableOf,drink liquids': 'yes',
 'HasPart,hair': 'yes',
 'IsA,air breathing vertebrate': 'yes',
 'CapableOf,mate': 'yes',
 'IsA,amniote': 'yes',
 'IsA,candy': 'no',
 'IsA,cheese': 'no',
 'IsA,salad': 'no',
 'IsA,sugar': 'no',
 'IsA,cake': 'no',
 'IsA,fruit': 'no',
 'IsA,meal': 'no',
 'IsA,soup': 'no',
 'IsA,vegetable': 'no',
 'IsA,fish': 'no',
 'IsA,bird': 'no',
 'IsA,reptile': 'no',
 'IsA,herb': 'no',
 'MadeOf,cells molecules and atoms': 'yes',
 'IsA,living creature': 'yes',
 'IsA,organism': 'yes',
 'CapableOf,eating': 'yes',
 'CapableOf,live': 'yes',
 'HasPart,face': 'yes',
 'IsA,living organism': 'yes',
 'HasProperty,alive': 'yes',
 'CapableOf,move itself about': 'yes',
 'HasPart,head': 'yes',
 'CapableOf,fight for life': 'yes',
 'CapableOf,note food': 'yes',
 'IsA,bacterium': 'no',
 'IsA,virus': 'no',
 'IsA,fungus': 'no',
 'IsA,plant': 'no',
 'Is

In [41]:
entity = 'american bison'
relation = 'IsA,plant'
label = facts[entity][relation]
true = (label == 'yes')

base = (entity, relation, true)
base

('american bison', 'IsA,plant', False)

In [55]:
random.sample(list(facts[entity].items()), 10)

[('IsA,science', 'no'),
 ('IsA,grass', 'no'),
 ('IsA,expert', 'no'),
 ('IsA,politician', 'no'),
 ('IsA,student', 'no'),
 ('IsA,room', 'no'),
 ('IsA,living creature', 'yes'),
 ('IsA,rodent', 'no'),
 ('IsA,herb', 'no'),
 ('IsA,human being', 'no')]

In [6]:
# TODO: add depth for deeper iteration of constraints tree
def find_constraints(constraints, filter_dict={}):
    def filter_fn(link):
        for k, v in filter_dict.items():
            if link[k] != v:
                return False
        return True
    
    return list(filter(filter_fn, constraints['links']))

In [45]:
if true:
    filter_dict = {
        'source': relation,
        'direction': 'forward',
    }
    selected_constraints = find_constraints(constraints, filter_dict=filter_dict)
else:
    filter_dict = {
        'weight': 'yes_yes',
        'target': relation,
        'direction': 'forward',
    }
    selected_constraints = find_constraints(constraints, filter_dict=filter_dict)

In [46]:
selected_constraints

[{'weight': 'yes_yes',
  'direction': 'forward',
  'score': 10,
  'source': 'IsA,tree',
  'target': 'IsA,plant'},
 {'weight': 'yes_yes',
  'direction': 'forward',
  'score': 10,
  'source': 'IsA,grass',
  'target': 'IsA,plant'},
 {'weight': 'yes_yes',
  'direction': 'forward',
  'score': 10,
  'source': 'IsA,fruit',
  'target': 'IsA,plant'},
 {'weight': 'yes_yes',
  'direction': 'forward',
  'score': 10,
  'source': 'IsA,flower',
  'target': 'IsA,plant'},
 {'weight': 'yes_yes',
  'direction': 'forward',
  'score': 10,
  'source': 'IsA,rose',
  'target': 'IsA,plant'},
 {'weight': 'yes_yes',
  'direction': 'forward',
  'score': 10,
  'source': 'IsA,vegetable',
  'target': 'IsA,plant'},
 {'weight': 'yes_yes',
  'direction': 'forward',
  'score': 10,
  'source': 'IsA,herb',
  'target': 'IsA,plant'}]

In [47]:
contradictions = []
for c in selected_constraints:
    contra = (entity, c['target'] if true else c['source'], true != (c['weight'] == 'yes_yes'))
    contradictions.append(contra)
print("Base:", base)
contradictions

Base: ('american bison', 'IsA,plant', False)


[('american bison', 'IsA,tree', True),
 ('american bison', 'IsA,grass', True),
 ('american bison', 'IsA,fruit', True),
 ('american bison', 'IsA,flower', True),
 ('american bison', 'IsA,rose', True),
 ('american bison', 'IsA,vegetable', True),
 ('american bison', 'IsA,herb', True)]

In [20]:
generate_assertion(*base)

'A american bison is a mammal.'

In [21]:
generate_question(*base)

('Is a american bison a mammal?', 'Yes')